<a href="https://colab.research.google.com/github/HansHenseler/DFRWS-APAC-LLM-Workshop/blob/main/Part_IV_Retrieval_Augmented_Generation_demo_with_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

****************************************************
####################################################

**Stage 1:
Download a PDF file and create vector store**
####################################################
****************************************************

In [8]:
# GET GPU FIRST !!!!!
# T4 GPU

# first we download a/some pdf files
# For simplicity, we just download and process one pdf file. You can create a folder and place your pdf files there.

!wget -O criminal_procedure_code.pdf https://sso.agc.gov.sg/Act/CPC2010?ViewType=Pdf&_=20230926140009

--2023-10-14 09:32:04--  https://sso.agc.gov.sg/Act/CPC2010?ViewType=Pdf
Resolving sso.agc.gov.sg (sso.agc.gov.sg)... 199.184.145.21
Connecting to sso.agc.gov.sg (sso.agc.gov.sg)|199.184.145.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1574942 (1.5M) [application/pdf]
Saving to: ‘criminal_procedure_code.pdf’

criminal_procedure_ 100%[===================>]   1.50M  1.43MB/s    in 1.0s    

2023-10-14 09:32:07 (1.43 MB/s) - ‘criminal_procedure_code.pdf’ saved [1574942/1574942]



In [9]:
!ls -l

total 9244
-rw-r--r-- 1 root root 1574942 Oct  5 09:58 'CPC2010?ViewType=Pdf'
-rw-r--r-- 1 root root 1574942 Oct  5 09:58 'CPC2010?ViewType=Pdf.1'
-rw-r--r-- 1 root root 1574942 Oct  5 09:58 'CPC2010?ViewType=Pdf.2'
-rw-r--r-- 1 root root 1574942 Oct  5 09:58 'CPC2010?ViewType=Pdf.3'
-rw-r--r-- 1 root root 1574942 Oct  5 09:58 'CPC2010?ViewType=Pdf.4'
-rw-r--r-- 1 root root 1574942 Oct  5 09:58  criminal_procedure_code.pdf
drwxr-xr-x 1 root root    4096 Oct 12 13:25  sample_data


In [10]:
!pip install langchain==0.0.310
!export LLAMA_CUBLAS=1
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python==0.2.11
!pip install sentence-transformers==2.2.2
!pip install chromadb==0.3.26
!pip install unstructured==0.10.19 pdf2image==1.16.3 pdfminer==20191125
!pip install glob2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-manylinux_2_35_x86_64.whl size=6423419 sha256=7d73c494eddd33d6ec3b24d1d596b03fbe3d984b3dd451a04f108f372fe95218
  Stored in directory: /root/.cache/pip/wheels/dc/42/77/a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Prepa

In [12]:
!pip uninstall pdfminer
!pip install pdfminer.six
!pip install unstructured==0.10.19
#from pdfminer.pdfinterp import PDFResourceManager
#from pdfminer.converter import PDFPageAggregator


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -f
  Using cached pdfminer.six-20221105-py3-none-any.whl (5.6 MB)


In [13]:
import chromadb
from typing import List
from chromadb.config import Settings
from langchain.document_loaders import (
    TextLoader,
    UnstructuredPDFLoader,

)

LOADER_MAPPING = {
    ".txt": (TextLoader, {}),       # text loader class, no parameter
    ".pdf": (UnstructuredPDFLoader, {}),      # PDF loader class, no parameter

}

In [14]:
# Load one document and save to a document object
from langchain.docstore.document import Document
import glob

def load_one_document(file_path):
  # get file extension
  file_ext = "." + file_path.rsplit(".", 1)[-1]

  # check whether the file type is in LOADER_MAPPING
  #   if YES, Load the file content to "Document" object
  #   if NO, skip
  docs_data = None
  if file_ext in LOADER_MAPPING:
    loader_class, loader_args = LOADER_MAPPING[file_ext]
    loader = loader_class(file_path, **loader_args)
    docs_data = loader.load()
  return docs_data


# Load all documents
import os
def load_all_documents(source_dir):
  all_files = glob.glob(source_dir + '/' + '*.*')
  all_docs_data = []
  for one_file in all_files:
    all_docs_data.extend(load_one_document(one_file))
  return all_docs_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

All files are now loaded into document objects. The next step is to split the contents hold in document objects into chunks (smaller pieces of text).

In [18]:
# source_directry is the path where input files are there
from langchain.text_splitter import RecursiveCharacterTextSplitter

source_directory = "/content"
chunk_size = 1000
chunk_overlap = 100

# load files and split contents
all_docs_data = load_all_documents(source_directory)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunked_docs = text_splitter.split_documents(all_docs_data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [19]:
chunked_docs[110]

Document(page_content='Form of search warrant 26.—(1) A search warrant issued by a court under this Code must be in writing bearing the seal of the court, and signed by a Magistrate or District Judge (as the case may be), or in the case of the General Division of the High Court, by a Judge or by the Registrar of the Supreme Court.\n\n[40/2019]\n\nInformal Consolidation – version in force from 26/6/2023\n\n2020 Ed.\n\nCriminal Procedure Code 2010\n\n52\n\n(2) A search warrant must ordinarily be issued to the Commissioner of Police and to one or more other police officers to be designated by name in the warrant, and all or any of those police officers may execute it.\n\n(3) The court may in appropriate circumstances issue a search warrant to one or more named persons who are not police officers, and all or any of those persons may execute it.', metadata={'source': '/content/criminal_procedure_code.pdf'})

In [20]:
# Create embedding function

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from chromadb.config import Settings


# configuration of the vector store
persist_directory = "./"

CHROMA_SETTINGS = Settings(
    chroma_db_impl = 'duckdb+parquet',  # use duckdb to r/w parquet file
    anonymized_telemetry = False     # refuse telemetry of usage info
)
model_name = "sentence-transformers/all-MiniLM-L12-v2"
model_kwargs = {'device': 'cuda'}      # you must have a gpu, otherwise change it to cpu
encode_kwargs = {'normalize_embeddings': True}
embedding_func = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

print("Creating new vector store...")
db = Chroma.from_documents(
    chunked_docs,
    embedding_func,
    persist_directory=persist_directory,
    client_settings=CHROMA_SETTINGS
)

db.persist()
db = None
print("Files ingestion completed!")

Creating new vector store...
Files ingestion completed!


In [21]:
!ls -l

total 4716
-rw-r--r-- 1 root root     557 Oct 14 09:49 chroma-collections.parquet
-rw-r--r-- 1 root root 3238754 Oct 14 09:49 chroma-embeddings.parquet
-rw-r--r-- 1 root root 1574942 Oct  5 09:58 criminal_procedure_code.pdf
drwxr-xr-x 2 root root    4096 Oct 14 09:49 index
drwxr-xr-x 1 root root    4096 Oct 12 13:25 sample_data


In [22]:
ls -l index

total 2076
-rw-r--r-- 1 root root   38968 Oct 14 09:49 id_to_uuid_6ab0e76a-248d-4e32-bee2-adc7c927c755.pkl
-rw-r--r-- 1 root root 2029800 Oct 14 09:49 index_6ab0e76a-248d-4e32-bee2-adc7c927c755.bin
-rw-r--r-- 1 root root     105 Oct 14 09:49 index_metadata_6ab0e76a-248d-4e32-bee2-adc7c927c755.pkl
-rw-r--r-- 1 root root   45553 Oct 14 09:49 uuid_to_id_6ab0e76a-248d-4e32-bee2-adc7c927c755.pkl


****************************************************
####################################################

**Stage 2:
Input files ingested and chromadb is created!
Now we go to the inference stage.**
####################################################
****************************************************

In [23]:
# This is the chat model!
!wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf

--2023-10-14 09:49:57--  https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.118, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/8d/b1/8db1d1f73b4caa58e947ccbfe2fb27ac5e495c2ad8457ad299d15987aee3b520/eda2a15d532bea4ce6fc14d15c2b72638243396816252f73a94dceeb0429112f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-13b-chat.Q4_0.gguf%3B+filename%3D%22llama-2-13b-chat.Q4_0.gguf%22%3B&Expires=1697536197&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NzUzNjE5N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84ZC9iMS84ZGIxZDFmNzNiNGNhYTU4ZTk0N2NjYmZlMmZiMjdhYzVlNDk1YzJhZDg0NTdhZDI5OWQxNTk4N2FlZTNiNTIwL2VkYTJhMTVkNTMyYmVhNGNlNmZjMTRkMTVjMmI3MjYzODI0M

In [ ]:
from langchain.llms import LlamaCpp
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    n_ctx = 4096,
    model_path="/content/llama-2-13b-chat.Q4_0.gguf", # location of the model
    temperature=0.75,                 # temperature
    max_tokens=2000,                 # Max. number of tokens to be generated
    top_p=0.9,                    # top_p = 0.9
    top_k=30,                     # top_k = 30
    n_gpu_layers=43,                 # number of layers to offload to GPU
    verbose=True, # Verbose is required to pass to the callback manager
    n_batch=200,          # number of token generation in parallel
)

In [25]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "You are a helpful, respectful and honest assistant."

def create_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT):
  SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
  prompt = B_INST + SYSTEM_PROMPT + instruction + E_INST
  return "<s>" + prompt

In [26]:
# open the vector database
from langchain.embeddings import HuggingFaceEmbeddings
from chromadb.config import Settings
from langchain.vectorstores import Chroma


model_name = "sentence-transformers/all-MiniLM-L12-v2"
model_kwargs = {'device': 'cuda'}      # you must have a gpu, otherwise change it to cpu
encode_kwargs = {'normalize_embeddings': True}
embedding_func = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)


persist_directory = "./"

CHROMA_SETTINGS = Settings(
    chroma_db_impl = 'duckdb+parquet',  # use duckdb to r/w parquet file
    anonymized_telemetry = False     # refuse telemetry of usage info
)

vector_db = Chroma(persist_directory=persist_directory,
            embedding_function=embedding_func,
            client_settings=CHROMA_SETTINGS)

In [27]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


instruction = """Given the following passage, you find out the answer to the question.

Passage: {context}

Question: {question}

Helpful answer: """

prompt_template = create_prompt(instruction)
PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

chain_type_kwargs = {"prompt": PROMPT}
retriever = vector_db.as_retriever(search_kwargs={"k": 5})

query = "what are different the sentences in Singapore?"
relevant_docs = retriever.get_relevant_documents(query)
print(relevant_docs)

[Document(page_content='(d) if, having changed his or her residence to a place outside Singapore, the person later returns to Singapore, personally present himself or herself and notify his or her return and his or her place of residence in Singapore to the officer in charge of the police division in which his or her residence in Singapore is situated; and\n\n(e) if the person intends to be absent from his or her last notified residence for more than 48 hours without changing his or her place of residence, personally present himself or herself and notify his or her intention, where he or she intends to go and how long he or she will be away to the officer in charge of the police division in which his or her residence is situated.', metadata={'source': '/content/criminal_procedure_code.pdf'}), Document(page_content='[19/2018; 40/2019]\n\n(2) Where a person 30 years of age or above —\n\n(a) is convicted before the General Division of the High Court or a District Court of an offence punis

In [28]:
# Q&A Method 1 : Use langchain RetrievalQA chain

qa_chain = RetrievalQA.from_chain_type(
                          llm=llm,
                          chain_type="stuff",
                          retriever=retriever,
                          return_source_documents=True,
                          chain_type_kwargs=chain_type_kwargs,
                      )
answer = qa_chain(query)
print("Question: ", answer['query'] + '\n\n')
print("Answer:", answer['result'] + '\n\n')
answer['source_documents']

Question:  what are different the sentences in Singapore?


Answer:   In Singapore, there are several types of sentences that can be imposed on individuals who have been convicted of a crime. The type of sentence will depend on the severity of the offense and the circumstances of the case. Here are some of the different sentences that can be imposed in Singapore:

1. Fines: Fines are common penalties for less serious crimes such as traffic offenses, minor assaults, and other non-violent offenses. The amount of the fine will depend on the severity of the crime and the individual's financial means.
2. Imprisonment: For more serious crimes such as robbery, drug offenses, and violent crimes, imprisonment is a common sentence. The length of the




[Document(page_content='(d) if, having changed his or her residence to a place outside Singapore, the person later returns to Singapore, personally present himself or herself and notify his or her return and his or her place of residence in Singapore to the officer in charge of the police division in which his or her residence in Singapore is situated; and\n\n(e) if the person intends to be absent from his or her last notified residence for more than 48 hours without changing his or her place of residence, personally present himself or herself and notify his or her intention, where he or she intends to go and how long he or she will be away to the officer in charge of the police division in which his or her residence is situated.', metadata={'source': '/content/criminal_procedure_code.pdf'}),
 Document(page_content='[19/2018; 40/2019]\n\n(2) Where a person 30 years of age or above —\n\n(a) is convicted before the General Division of the High Court or a District Court of an offence puni

In [29]:
myquery="what is the sentence for burglary?"
answer = qa_chain(myquery)
print("Question: ", answer['query'] + '\n\n')
print("Answer:", answer['result'] + '\n\n')
answer['source_documents']

ValueError: ignored